In [1]:
from utils.config import *
from agents.trainer import Trainer
from utils.metrics import *

import torchaudio
torchaudio.set_audio_backend("soundfile")


# JSON = 'configs/VGGVox.json'
# JSON = 'configs/pre_trained/AutoSpeech.json'
JSON = 'configs/pre_trained/ResNetSE34L.json'
# JSON = 'configs/pre_trained/Cnn3D.json'
# JSON = 'configs/pre_trained/SincNet.json'
# JSON = 'configs/pre_trained/MobileNet.json'

config = process_config(JSON)
trainer = Trainer(config)

[INFO]: Experiment: ResNetSE34L Pretrained.
[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Current cuda device  0


In [5]:
import importlib

def load_model():
    Model = importlib.import_module(
        'graphs.models.' + config.model).__getattribute__(config.model)
    config.device = torch.device("cuda")
    return Model(**vars(config)).to(config.device)

model = load_model()

In [12]:
model = trainer.__model__
for e in trainer.test_loader:
    break
input_size = e[0].shape

In [13]:
# compute_complexity(model, input_size)
compute_inference_time(model, input_size)

DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING
DONE WITH DRY RUNS, NOW BENCHMARKING


([26.58517360687256,
  25.135886669158936,
  24.561291933059692,
  23.823833465576172,
  23.480498790740967,
  23.352233320474625,
  23.269422724843025],
 [0.49052400990603756,
  0.22940952626754413,
  0.11349339005836843,
  0.07998310370885635,
  0.05218680078123108,
  0.03432211083643411,
  0.014535861196974588])